In [15]:
import pandas as pd
import numpy as np
import re
import nltk
import sys
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.regularizers import l2
from nltk.stem import PorterStemmer
from keras.layers import TimeDistributed

#import statments 

In [2]:
data = pd.read_csv('/Users/davidan/Downloads/fade-net/data/data_shuffled.csv')
data['bodytext'] = data['bodytext'].str.lower()

def identify_tokens(row):
    news = row['bodytext']
    tokens = nltk.word_tokenize(news)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

data['words'] = data.apply(identify_tokens, axis=1)

stops = set(stopwords.words("english"))                  

def remove_stops(row):
    my_list = row['words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

data['stem_meaningful'] = data.apply(remove_stops, axis=1)

def rejoin_words(row):
    my_list = row['stem_meaningful']
    joined_words = ( " ".join(my_list))
    return joined_words

data['processed'] = data.apply(rejoin_words, axis=1)

# cols_to_drop = ['words', 'stem_meaningful']
# data.drop(cols_to_drop, inplace=True)

data['bodytext'] = data['processed']


data.head()

KeyboardInterrupt: 

In [21]:
dropcol = ['words', 'stem_meaningful', 'processed']
data.drop(labels = dropcol, axis = 1)

data.to_csv('/Users/davidan/Downloads/fade-net/data/tokenizedData.csv', index = False)

#manually check

In [ ]:
data = pd.read_csv('/Users/davidan/Downloads/fade-net/data/tokenizedData.csv')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data.head()

In [3]:
data = pd.read_csv('/Users/davidan/Downloads/fade-net/data/tokenizedData.csv')


news = data['bodytext'].values
label = data['label'].values

#delete the header file 
news = np.delete(news, (4396), axis=0)
label = np.delete(label, (4396), axis=0)

counter_news = 0

for entry in news:
    news[counter_news] = news[counter_news][:244]

label = np.array(label)

label = label.astype(np.float)



print(label)
print(type(label[2]))

[0. 1. 0. ... 0. 1. 1.]
<class 'numpy.float64'>


In [4]:
print(news[0])

guardian survey found pervasive culture staff bullying parts detrimental impact staff wellbeing patient stories professionals submitted survey guardian diagnosed acute stress incident critically high blood frightened one man thought nothing sex


In [5]:
t = Tokenizer()
t.fit_on_texts(news)

text_matrix = t.texts_to_sequences(news)

print('News: ' + news[0])
print('Representation: ')
print(text_matrix[0])

#find max length of vectors

max_length = 0
min_length = 1
total_length = 0

emb_dim = 16

vocab_size = len(t.word_index) + 1

for i in range(len(text_matrix)):
    sent_length = len(text_matrix[i])
    if max_length < sent_length:
        max_length = sent_length
        
for i in range(len(text_matrix)):
    sent_length = len(text_matrix[i])
    if min_length > sent_length:
        min_length = sent_length

for i in range(len(text_matrix)):
    sent_length = len(text_matrix[i])
    total_length += sent_length

print('Average Length %d' % int(total_length/19255))
print('vocabulary size: %d'%vocab_size)
print('Max Length: %d' % max_length)
print('Min Length: %d' % min_length)

print(type(text_matrix[1]))

News: guardian survey found pervasive culture staff bullying parts detrimental impact staff wellbeing patient stories professionals submitted survey guardian diagnosed acute stress incident critically high blood frightened one man thought nothing sex
Representation: 
[413, 2105, 92, 8134, 648, 444, 5276, 991, 9492, 532, 444, 5702, 2753, 600, 3014, 4103, 2105, 413, 4981, 5787, 2292, 1315, 7115, 146, 1053, 6897, 2, 97, 228, 188, 911]
Average Length 244
vocabulary size: 48543
Max Length: 2524
Min Length: 1
<class 'list'>


In [6]:
print(text_matrix[0])

[413, 2105, 92, 8134, 648, 444, 5276, 991, 9492, 532, 444, 5702, 2753, 600, 3014, 4103, 2105, 413, 4981, 5787, 2292, 1315, 7115, 146, 1053, 6897, 2, 97, 228, 188, 911]


In [7]:
#split datasets 
#No need to post pad data since all have lengths 
text_matrix = pad_sequences(text_matrix, maxlen=max_length, padding='post')

# text_matrix = np.array(text_matrix, dtype=None)
# label = np.array(label, dtype= None)

X_train, X_test, y_train, y_test = train_test_split(text_matrix, label, test_size = 0.33, random_state = 53)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [8]:
print(type(y_train))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [7]:
from tensorflow.keras.regularizers import l2

lstm_units = 12
inputs = Input(shape = (64,))
embedding = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=244, embeddings_regularizer=l2(.001))
embd_out = embedding(inputs)

lstm = LSTM(lstm_units, dropout = 0.25, recurrent_dropout = 0.2)
lstm_out = lstm(embd_out)

prob = Dense(1, activation = 'relu')
outputs = prob(lstm_out)

model = Model(inputs, outputs)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 64, 16)            776688    
_________________________________________________________________
lstm (LSTM)                  (None, 12)                1392      
_________________________________________________________________
dense (Dense)                (None, 1)                 13        
Total params: 778,093
Trainable params: 778,093
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.fit(x=X_train,y=y_train,
          batch_size=80,
          epochs=10,
          verbose=1,
          shuffle=True,
          validation_data=(X_test,y_test)
         )

Epoch 1/10
164/164 [==============================] - 508s 3s/step - loss: 7.6728 - acc: 0.5058 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 2/10
164/164 [==============================] - 440s 3s/step - loss: 7.7659 - acc: 0.4965 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 3/10
164/164 [==============================] - 402s 2s/step - loss: 7.6853 - acc: 0.5018 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 4/10
164/164 [==============================] - 405s 2s/step - loss: 7.7203 - acc: 0.4995 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 5/10
164/164 [==============================] - 415s 3s/step - loss: 7.7882 - acc: 0.4951 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 6/10
164/164 [==============================] - 402s 2s/step - loss: 7.6755 - acc: 0.5024 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 7/10
164/164 [==============================] - 375s 2s/step - loss: 7.7031 - acc: 0.5006 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 8/10
164/164 [==============================] - 370s 2s/step - 

In [8]:
model.save("model")

INFO:tensorflow:Assets written to: model/assets


In [10]:
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer

In [11]:
class AttentionMech(Layer):
    def __init__(self, **kwargs):
        super(AttentionMech, self).__init__(**kwargs)
    
    # This method states the weights that the layer will learn. It has as input param the shape of the input
    # which is called. This method is called at the declaration time
    def build(self, input_shape):
        # We need to provide the dimensions of our weights. In this example, we will have a W_a matrix of
        # dimension (lstm_units, 1), and a bias of dimension (max_length, 1)
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(AttentionMech, self).build(input_shape)
    
    # In this method with do all the calculations of the layer and return the output of the layer
    def call(self, x):
        # x is the input of the layer. In this example, the output of lstm (hidden_statesxlstm_units) 
        # hidden_states = max_length
        
        # We calculate the score tanh(W.x + b)
        scores = K.tanh(K.dot(x,self.W)+self.b)  # (max_length x 1) 
        print('scores shape: ')
        print(scores.shape)
        
        # This removes the last axis -> a vector of max_length dimension 
        # we can omit this since our W matrix has dimension 1 in the last axis
        scores=K.squeeze(scores, axis=-1) 
        print('scores shape after squeeze: ')
        print(scores.shape)
        
        # we apply softmax (the last axis is the default axis used for calculation)
        at=K.softmax(scores)
        print('attention weights shape: ')
        print(at.shape)
        
        # This adds a 1-sized dimension to the last axis -> matrix of (max_length x 1)
        at=K.expand_dims(at,axis=-1) # if there is no squeeze, then we can omit this
        print('attention weights shape after expand_dims: ')
        print(at.shape)
        
        # We calculate the weighted values -> \alpha*hidden_states         
        # row-wise multiplication (we are weighting the hidden_states, not the lstm_units) 
        output=x*at # (max_length x lstm_units)
        print('weighted values shape: ')
        print(output)
        
        # The output of this layer is the weighted values (we sum up the values of the hidden states), and
        # the weights of the attetnion (max_length x 1)
        return K.sum(output, axis=1), at
    
    # This is used for summary, to see the output shape of the two output matrices
    def compute_output_shape(self, input_shape):
        return (input_shape[0],input_shape[-1])
    
    # This is used for summary (it returns the params of the layer)
    def get_config(self):
        return super(BahdanauAttention, self).get_config()

In [13]:
from tensorflow.keras.layers import Attention, GlobalAveragePooling1D

# Architecture
inputs1 = Input(shape=(max_length,))
embedding1 = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=max_length, embeddings_regularizer=l2(.001))
embd_out1 = embedding1(inputs1)
lstm1 = LSTM(12, dropout=0.3, recurrent_dropout=0.2, return_sequences=True)
lstm_out1 = lstm1(embd_out1)

# attention = GlobalAveragePooling1D(Attention()([lstm_out1, lstm_out1]))
weigthed_out, weights = AttentionMech()(lstm_out1)

prob1 = Dense(1, activation='sigmoid')
outputs1 = prob1(weigthed_out)

model1 = Model(inputs1, outputs1) # classifier
attention_model = Model(inputs1, weights) # attention weights


print(model1.summary())

scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("attention_mech/mul:0", shape=(None, 2524, 12), dtype=float32)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2524)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 2524, 16)          776688    
_________________________________________________________________
lstm (LSTM)                  (None, 2524, 12)          1392      
_________________________________________________________________
attention_mech (AttentionMec ((None, 12), (None, 2524, 2536      
_________________________________________________________________
dense (Dense)                (None, 1)                 13  

In [35]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model1.fit(x=X_train,y=y_train,
          batch_size=80,
          epochs=10,
          verbose=1,
          shuffle=True,
          validation_data=(X_test,y_test)
          )

Epoch 1/10
scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("model_1/attention_mech_1/mul:0", shape=(None, 2524, 12), dtype=float32)
scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("model_1/attention_mech_1/mul:0", shape=(None, 2524, 12), dtype=float32)
164/164 [==============================] - ETA: 0s - loss: 0.8675 - acc: 0.5079scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("model_1/attention_mech_1/mul:0", shape=(None, 2524, 12), dtype=float32)
164/164 [==============================] - 435s 3s/step - loss: 0.8668 - acc: 0.50

In [40]:
from tensorflow.keras.layers import Attention, GlobalAveragePooling1D

# Architecture
inputs1 = Input(shape=(max_length,))
embedding1 = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=max_length, embeddings_regularizer=l2(.001))
embd_out1 = embedding1(inputs1)
lstm1 = Bidirectional(LSTM(lstm_units, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))
lstm_out1 = lstm1(embd_out1)

# attention = GlobalAveragePooling1D(Attention()([lstm_out1, lstm_out1]))
weigthed_out, weights = AttentionMech()(lstm_out1)

prob1 = Dense(1, activation='sigmoid')
outputs1 = prob1(weigthed_out)

model1 = Model(inputs1, outputs1) # classifier
attention_model = Model(inputs1, weights) # attention weights


print(model1.summary())

scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("attention_mech_4/mul:0", shape=(None, 2524, 24), dtype=float32)
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 2524)]            0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 2524, 16)          776688    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 2524, 24)          2784      
_________________________________________________________________
attention_mech_4 (AttentionM ((None, 24), (None, 2524, 2548      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 

In [18]:
import tensorflow.keras.metrics

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.TruePositives(), 
                                                                      tf.keras.metrics.Precision(),
                                                                      tf.keras.metrics.TrueNegatives(),
                                                                      tf.keras.metrics.FalseNegatives(),
                                                                      tf.keras.metrics.FalsePositives(), 'acc'])
model1.fit(x=X_train,y=y_train,
          batch_size=80,
          epochs=10,
          verbose=1,
          shuffle=True,
          validation_data=(X_test,y_test)
          )

Epoch 1/10
scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("model/attention_mech/mul:0", shape=(None, 2524, 12), dtype=float32)
scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("model/attention_mech/mul:0", shape=(None, 2524, 12), dtype=float32)
164/164 [==============================] - ETA: 0s - loss: 0.7485 - true_positives_1: 2494.0854 - precision_1: 0.5055 - true_negatives_1: 846.5976 - false_negatives_1: 817.0000 - false_positives_1: 2441.8598 - acc: 0.5062scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("model/attention_mech

In [19]:
import tensorflow.keras.metrics

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.TruePositives(), 
                                                                      tf.keras.metrics.Precision(),
                                                                      tf.keras.metrics.TrueNegatives(),
                                                                      tf.keras.metrics.FalseNegatives(),
                                                                      tf.keras.metrics.FalsePositives(), 'acc'])
model1.fit(x=X_train,y=y_train,
          batch_size=80,
          epochs=15,
          verbose=1,
          shuffle=True,
          validation_data=(X_test,y_test)
          )

Epoch 1/15
scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("model/attention_mech/mul:0", shape=(None, 2524, 12), dtype=float32)
scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("model/attention_mech/mul:0", shape=(None, 2524, 12), dtype=float32)
164/164 [==============================] - ETA: 0s - loss: 0.3778 - true_positives_2: 2987.1402 - precision_2: 0.8536 - true_negatives_2: 2793.6220 - false_negatives_2: 295.4085 - false_positives_2: 523.3720 - acc: 0.8777scores shape: 
(None, 2524, 1)
scores shape after squeeze: 
(None, 2524)
attention weights shape: 
(None, 2524)
attention weights shape after expand_dims: 
(None, 2524, 1)
weighted values shape: 
Tensor("model/attention_mech

In [ ]:
]